In [1]:
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=3)
search.invoke("Who are the top of the IPL teams currently in the table?")

[{'url': 'https://www.hindustantimes.com/cricket/ipl/points-table',
  'content': 'Royal Challengers Bengaluru (RCB) and Gujarat Titans (GT) occupy the second and third spots, respectively, each with two wins and a healthy net run rate. Rajasthan Royals’ win over PBKS lifted them to seventh, while Lucknow Super Giants (LSG), despite a thrilling victory over Mumbai Indians (MI), remained in sixth place. Kolkata Knight Riders with two wins are placed fifth in the IPL 2025 standings. [...] IPL Points Table: Delhi Capitals (DC) have surged to the top of the IPL 2025 points table after a convincing win over Chennai Super Kings (CSK) on Saturday. With three wins from as many matches, DC have looked like the most balanced side so far, boasting a net run rate of +1.257. Punjab Kings (PBKS), who suffered their first defeat of the season at the hands of Rajasthan Royals (RR), have dropped to fourth in the standings. [...] In IPL 2024, the final team standings at the end of the league stage saw Ko

In [5]:
tools =[search]

In [6]:
tools
print(tools.count)

<built-in method count of list object at 0x000001ACDE213600>


In [7]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)


In [10]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Who are the top of the IPL teams currently in the table?")]})

response["messages"]

[HumanMessage(content='Who are the top of the IPL teams currently in the table?', id='380922d2-664b-4f03-a550-d3e7077c02a1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5brDt6RsdSB8Q0PlL2rFXJdb', 'function': {'arguments': '{"query":"IPL 2021 points table"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 94, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0ccf1e43-66bc-49e7-ab41-c2cf826333c4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'IPL 2021 points table'}, 'id': 'call_5brDt6RsdSB8Q0PlL2rFXJdb', 'type': 'tool_call'}], usage_metadata={'input_t

In [11]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="what is Python?")]})

response["messages"]

[HumanMessage(content='what is Python?', id='c1a4c760-8d3d-4eb9-a3b5-b539495ad768'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JDVvgXK0fb6TzaVcu6Vwe3Ne', 'function': {'arguments': '{"query":"Python"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 85, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fbf5e1d1-8e4e-422b-85c0-d3543e150303-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Python'}, 'id': 'call_JDVvgXK0fb6TzaVcu6Vwe3Ne', 'type': 'tool_call'}], usage_metadata={'input_tokens': 85, 'output_tokens': 19, 'total_tokens': 104}),
 ToolMessage(co

In [12]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who are the top of the IPL teams currently in the table?")]}
    ): 
        print(chunk)
        print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zcn0jNAEYrTGfCrJRzI0OsFR', 'function': {'arguments': '{"query":"IPL 2021 current points table"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 94, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b8c66ba0-87c5-4cfa-bb7e-b120c0f964b6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'IPL 2021 current points table'}, 'id': 'call_zcn0jNAEYrTGfCrJRzI0OsFR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 26, 'total_tokens': 120})]}}
----
{'tools': {'messages': [

## Adding Memory

In [13]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [22]:
agent_executor = create_react_agent(llm , tools, checkpointer=memory)
config ={"configurable": {"thread_id": "001"}}

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who are the top of the IPL teams currently in the table?")]}, config
    ): 
        print(chunk)
        print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7ONNQPtnbexRdXQaRHXNAIF2', 'function': {'arguments': '{"query":"IPL 2021 points table"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 1986, 'total_tokens': 2011, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6e3630de-8064-4f0e-a631-e254fe84bb7a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'IPL 2021 points table'}, 'id': 'call_7ONNQPtnbexRdXQaRHXNAIF2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1986, 'output_tokens': 25, 'total_tokens': 2011})]}}
----
{'tools': {'messages': [ToolMessag

In [25]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="How many points top place team have now?")]}, config
    ): 
        print(chunk)
        print("----")

{'agent': {'messages': [AIMessage(content='The top-placed team, Delhi Capitals (DC), currently has 20 points in the IPL points table.', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 2650, 'total_tokens': 2674, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d004e3e9-d22c-4e4c-8004-fafa06085894-0', usage_metadata={'input_tokens': 2650, 'output_tokens': 24, 'total_tokens': 2674})]}}
----
